In [95]:
# -*- coding: utf-8 -*-
import ujson
import json
import pprint
import glob
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from operator import itemgetter
import multiprocessing
from copy import deepcopy
import psycopg2

from itertools import ifilter

pp = pprint.PrettyPrinter(indent=4)
%pylab inline

Populating the interactive namespace from numpy and matplotlib
time: 27.9 ms


In [3]:
%load_ext autotime

In [4]:
def log_progress(sequence, every=None, size=None):
    from ipywidgets import IntProgress, HTML, VBox
    from IPython.display import display

    is_iterator = False
    if size is None:
        try:
            size = len(sequence)
        except TypeError:
            is_iterator = True
    if size is not None:
        if every is None:
            if size <= 200:
                every = 1
            else:
                every = int(size / 200)     # every 0.5%
    else:
        assert every is not None, 'sequence is iterator, set every'

    if is_iterator:
        progress = IntProgress(min=0, max=1, value=1)
        progress.bar_style = 'info'
    else:
        progress = IntProgress(min=0, max=size, value=0)
    label = HTML()
    box = VBox(children=[label, progress])
    display(box)

    index = 0
    try:
        for index, record in enumerate(sequence, 1):
            if index == 1 or index % every == 0:
                if is_iterator:
                    label.value = '{index} / ?'.format(index=index)
                else:
                    progress.value = index
                    label.value = u'{index} / {size}'.format(
                        index=index,
                        size=size
                    )
            yield record
    except:
        progress.bar_style = 'danger'
        raise
    else:
        progress.bar_style = 'success'
        progress.value = index
        label.value = str(index or '?')


time: 32.1 ms


In [118]:
sport_lookup = {36: 'handball',
           34: 'rugby',
           29: 'cricket',
           44: 'beach_volleyball',
           89: 'skateboarding',
           25: 'baseball',
           48: 'martial_arts',
           45: 'volleyball',
           8: 'snowboarding',
           37: 'hockey',
           94: 'treadmill_walking',
           31: 'dancing',
           26: 'basketball',
           93: 'climbing',
           43: 'tennis',
           33: 'american_football',
           15: 'golf',
           27: 'boxing',
           10: 'kite_surfing',
           91: 'snowshoeing',
           28: 'stair_climing',
           47: 'yoga',
           92: 'wheelchair',
           87: 'circuit_training',
           38: 'pilates',
           7: 'downhill_skiing',
           30: 'elliptical',
           24: 'badminton',
           23: 'aerobics',
           5: 'roller_skiing',
           50: 'step_counter',
           12: 'sailing',
           13: 'windsurfing',
           88: 'treadmill_running',
           49: 'gymnastics',
           9: 'kayaking',
           17: 'orienteering',
           11: 'rowing',
           19: 'horseback_riding',
           35: 'soccer',
           46: 'weight_training',
           14: 'fitness_walking',
           16: 'hiking',
           21: 'indoor_cycling',
           22: 'core_stability_training',
           20: 'swimming',
           4: 'skate',
           0: 'run',
           40: 'scuba_diving',
           1: 'bike_transport',
           2: 'bike',
           3: 'mountain_bike',
           18: 'walk',
           41: 'squash',
           42: 'table_tennis',
           90: 'surfing',
           39: 'polo',
           32: 'fencing',
           86: 'walk_transport',
           84: 'weight_lifting',
           58: 'circuit_training',
           6: 'cross_country_skiing'}

time: 63.5 ms


In [5]:
!ls /Users/patrickmulrooney/Downloads/endomondo/individual_users/

10014612.txt 13619220.txt 2183651.txt  366212.txt   5346596.txt  7723530.txt
10027353.txt 1362730.txt  2207788.txt  3664548.txt  535315.txt   7726077.txt
10028940.txt 13637398.txt 2209215.txt  3668539.txt  5358123.txt  7734791.txt
10051240.txt 13653.txt    221371.txt   3672721.txt  5362316.txt  7737399.txt
10057948.txt 13657138.txt 2215571.txt  367324.txt   5378465.txt  7743096.txt
10086630.txt 13665835.txt 2218816.txt  3674362.txt  538524.txt   7748686.txt
1009668.txt  13668730.txt 22260.txt    3680369.txt  5386323.txt  7752286.txt
10099521.txt 1367691.txt  2226932.txt  368603.txt   5395086.txt  775277.txt
10099726.txt 1367925.txt  2227649.txt  3688079.txt  5407179.txt  77567.txt
10102190.txt 13681075.txt 2228208.txt  3689976.txt  5421474.txt  7761529.txt
10112432.txt 13681949.txt 2232554.txt  3691161.txt  5435152.txt  7763998.txt
10122140.txt 1368888.txt  2232580.txt  3692720.txt  5453063.txt  776483.txt
10132518.txt 13693003.txt 2237368.txt  3700284.txt  5466683.txt  7769983.txt
101

time: 26min 50s


In [92]:
print total_count
print weather_count

1320401
234921
time: 1.3 ms


In [93]:
j

{u'calories': 2466.0,
 u'distance': 62.3647,
 u'duration': 9902.863,
 u'heart_rate_avg': 141.0,
 u'heart_rate_max': 177.0,
 u'id': 189117508,
 u'speed_avg': 22.67151630796064,
 u'speed_max': 62.9496,
 u'sport': 'bike'}

time: 4.52 ms


In [129]:
dbname = 'endomondo'
conn = psycopg2.connect("dbname=endomondo user=patrickmulrooney")

# Open a cursor to perform database operations
cur = conn.cursor()


time: 13.5 ms


In [97]:
cur.execute("select relname from pg_class where relkind='r' and relname !~ '^(pg_|sql_)';")
series_tables = []
workout_tables = []
for _i in cur.fetchall():
    if 'by_workout' in _i[0]:
        workout_tables.append(_i[0])
        continue
        
    series_tables.append(_i[0])

time: 35.3 ms


In [103]:
for _table in workout_tables:
    query = "ALTER TABLE {} DROP COLUMN IF EXISTS altitude_max;".format(_table)
    cur.execute(query)
    query = "ALTER TABLE {} DROP COLUMN IF EXISTS altitude_min;".format(_table)
    cur.execute(query)
    query = "ALTER TABLE {} DROP COLUMN IF EXISTS calories;".format(_table)
    cur.execute(query)
    query = "ALTER TABLE {} DROP COLUMN IF EXISTS distance;".format(_table)
    cur.execute(query)
    query = "ALTER TABLE {} DROP COLUMN IF EXISTS duration;".format(_table)
    cur.execute(query)
    query = "ALTER TABLE {} DROP COLUMN IF EXISTS heart_rate_avg;".format(_table)
    cur.execute(query)
    query = "ALTER TABLE {} DROP COLUMN IF EXISTS heart_rate_max;".format(_table)
    cur.execute(query)
    query = "ALTER TABLE {} DROP COLUMN IF EXISTS hydration;".format(_table)
    cur.execute(query)
    query = "ALTER TABLE {} DROP COLUMN IF EXISTS speed_avg;".format(_table)
    cur.execute(query)
    query = "ALTER TABLE {} DROP COLUMN IF EXISTS speed_max;".format(_table)
    cur.execute(query)
    query = "ALTER TABLE {} DROP COLUMN IF EXISTS humidity;".format(_table)
    cur.execute(query)
    query = "ALTER TABLE {} DROP COLUMN IF EXISTS temperature;".format(_table)
    cur.execute(query)
    query = "ALTER TABLE {} DROP COLUMN IF EXISTS wind_speed;".format(_table)
    cur.execute(query)
    conn.commit()

time: 172 ms


In [104]:
for _table in workout_tables:
    query = "ALTER TABLE {} ADD COLUMN altitude_max numeric(20,10);".format(_table)
    cur.execute(query)
    query = "ALTER TABLE {} ADD COLUMN altitude_min numeric(20,10);".format(_table)
    cur.execute(query)
    query = "ALTER TABLE {} ADD COLUMN calories numeric(20,10);".format(_table)
    cur.execute(query)
    query = "ALTER TABLE {} ADD COLUMN distance numeric(20,10);".format(_table)
    cur.execute(query)
    query = "ALTER TABLE {} ADD COLUMN duration numeric(20,10);".format(_table)
    cur.execute(query)
    query = "ALTER TABLE {} ADD COLUMN heart_rate_avg numeric(20,10);".format(_table)
    cur.execute(query)
    query = "ALTER TABLE {} ADD COLUMN heart_rate_max numeric(20,10);".format(_table)
    cur.execute(query)
    query = "ALTER TABLE {} ADD COLUMN hydration numeric(20,10);".format(_table)
    cur.execute(query)
    query = "ALTER TABLE {} ADD COLUMN speed_avg numeric(20,10);".format(_table)
    cur.execute(query)
    query = "ALTER TABLE {} ADD COLUMN speed_max numeric(20,10);".format(_table)
    cur.execute(query)
    query = "ALTER TABLE {} ADD COLUMN humidity integer;".format(_table)
    cur.execute(query)
    query = "ALTER TABLE {} ADD COLUMN temperature integer;".format(_table)
    cur.execute(query)
    query = "ALTER TABLE {} ADD COLUMN wind_speed integer;".format(_table)
    cur.execute(query)
    conn.commit()

time: 146 ms


In [141]:
dbname = 'endomondo'
conn = psycopg2.connect("dbname=endomondo user=patrickmulrooney")

# Open a cursor to perform database operations
cur = conn.cursor()


endoFiles = glob.glob('/Users/patrickmulrooney/Downloads/endomondo/individual_users/*')

total_count = 0
weather_count = 0
count = 0

done = False
for _file in log_progress(endoFiles, every=1):
    count += 1
    
    if count not in [656, 1035]:
        continue
            
    if done:
        break
    skipped = False
    endo = []
    
    with open(_file, 'r') as f:
        lines = f.readlines()
        for _line in lines:
            if not skipped:
                skipped = True
                continue
            total_count += 1
            _line = _line.replace("'","\"")
            _line = _line.replace(" u\""," \"")
            _line = _line.replace("{u\"","{\"")
            _line = _line.replace("None","\"None\"")

            j = ujson.loads( _line )
            
            del j['heart_rate_zones']
            del j['start_time']
            del j['descent']
            del j['ascent']
            del j['points']
            
            for _j in j.keys():
                if j[_j] == 'None':
                    del j[_j]
                 
            workoutid = j['id']
            sport = sport_lookup[j['sport']]
            
            del j['sport']
            del j['id']
            
            if 'weather' in j.keys():
                if j['weather'].keys() != ['type']:
                    weather_count += 1
                    j['humidity'] = j['weather']['humidity']
                    j['temperature'] = j['weather']['temperature']
                    j['wind_speed'] = j['weather']['wind_speed']
                
                del j['weather']
                
            update_str = []
            
            try:
                if abs(j['altitude_max']) > 10000000000:
                    del j['altitude_max']
            except:
                pass
            try:
                if abs(j['altitude_min']) > 10000000000:
                    del j['altitude_min']
            except:
                pass
            
            for _k, _v in j.iteritems():
                update_str.append("%s = %s"%(_k,_v))
                
            update_str = str.join(", ",update_str)
            
            if update_str == "":
                continue
            
            query = 'update {}_by_workout set {} where workoutid = {}'.format(sport,update_str,workoutid)
            cur.execute(query)
            conn.commit()

time: 3min 42s


In [117]:
#656, 1035

'update cross-country_skiing_by_workout set distance = 15.1450004578,speed_max = 27.1993,calories = 1525.82,speed_avg = 7.03764870204,altitude_max = 378.9,duration = 7747.19,hydration = 0.77,altitude_min = 306.0 where workoutid = 487336203'

time: 3.45 ms


In [143]:
query

'update run_by_workout set heart_rate_max = 172.0, distance = 12.0789267578, heart_rate_avg = 152.0, calories = 898.0, altitude_max = 102.998, altitude_min = -0.343384 where workoutid = 3620355'

time: 5.33 ms
